In [232]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os

In [233]:
# Define main project folder
main = r'C:\Users\Do Thu An\OneDrive\Desktop\Dynamic Macroeconomics\Problem sets\Dynamic-Macroeconomics\PS2_Code'
# Set project folder as current working directory
os.chdir(main)

# Define data file path
data_path = os.path.join(main, 'Data Files', 'VHLSS 2008 Data')

In [234]:
# Load muc123a
muc123a = pd.read_csv(os.path.join(data_path, 'muc123a.csv'))

# Create household size column
muc123a['hsize'] = muc123a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['matv'].transform('max')

# Keep only household heads who are male and age ≥ 25
muc123a = muc123a[(muc123a['m1ac3'] == 1) & (muc123a['m1ac2'] == 1) & (muc123a['m1ac5'] >= 25)]

# Keep relevant columns
columns123a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv', 'hsize', 'm1ac2', 'm1ac3', 'm1ac5']
df = muc123a[columns123a].copy()

In [235]:
# Load and process income file
muc4a = pd.read_csv(os.path.join(data_path, 'muc4a.csv'))

# Create household size column
muc4a['hsize'] = muc4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['matv'].transform('max')

# Select the required columns
columns4a = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv', 'm4ac6', 'm4ac7', 'm4ac8', 'm4ac11', 
             'm4ac12f', 'm4ac16', 'm4ac17', 'm4ac18', 'm4ac21', 'm4ac22f', 'm4ac25']
muc4a = muc4a[columns4a]

# Define income-related columns and filter existing ones
income_cols = ['m4ac11', 'm4ac12f', 'm4ac21', 'm4ac22f', 'm4ac25']
income_cols = [col for col in income_cols if col in muc4a.columns]

# Compute individual income (handle missing columns)
muc4a['indi_income'] = muc4a[income_cols].sum(axis=1) if income_cols else 0

# Compute total household income
hh_income = muc4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['indi_income'].sum().reset_index()
hh_income.rename(columns={'indi_income': 'hh_income'}, inplace=True)

# Merge total household income
muc4a = muc4a.merge(hh_income, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Define work-related columns and filter existing ones
work_cols = ['m4ac6', 'm4ac7', 'm4ac8', 'm4ac16', 'm4ac17', 'm4ac18']
work_cols = [col for col in work_cols if col in muc4a.columns]

# Fill NaN values with 0 to avoid multiplication errors
muc4a[work_cols] = muc4a[work_cols].fillna(0)

# Compute total working hours for first and secondary jobs
muc4a['total_hours_first'] = muc4a['m4ac6'] * muc4a['m4ac7'] * muc4a['m4ac8']
muc4a['total_hours_second'] = muc4a['m4ac16'] * muc4a['m4ac17'] * muc4a['m4ac18']

# Compute total individual working hours
muc4a['indi_work_hours'] = muc4a['total_hours_first'] + muc4a['total_hours_second']

# Compute total household working hours
hh_work_hours = muc4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['indi_work_hours'].sum().reset_index()
hh_work_hours.rename(columns={'indi_work_hours': 'hh_work_hours'}, inplace=True)

# Merge total household working hours
muc4a = muc4a.merge(hh_work_hours, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

# Fill any remaining NaN values
muc4a.fillna(0, inplace=True)

# Merge with the main dataframe
df = df.merge(muc4a, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'], how='left')

In [236]:
def merge_wealth(df, file_name, columns, value_cols, new_col_name):
    # Load the wealth dataset with selected columns
    data = pd.read_csv(os.path.join(data_path, file_name))[columns]
    # Fill missing values with zero
    data.fillna(0, inplace=True)
    # Calculate household wealth
    if 'm6ac7' in data.columns:  # If percentage ownership column exists (for fixed assets)
        data['m6ac7'] = data['m6ac7'] / 100  # Convert percentage to decimal
        data[new_col_name] = data['m6ac3'] * data['m6ac6'] * data['m6ac7']
    else:  # For durable goods (no ownership percentage)
        data[new_col_name] = data['m6bc3'] * data['m6bc6']

    # Group by household to get total wealth per household
    wealth = data.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[new_col_name].sum().reset_index()

    # Merge aggregated wealth data into the main dataframe
    return df.merge(wealth, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

In [237]:
# Merge wealth data from fixed assets (6A)
# Define the required columns (household identifiers + selected variables)
# m6ac3: quantity of the assets, m6ac6: assets' value at current price, m6ac7: percentage of ownership
df = merge_wealth(df, 'muc6a.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6ac3', 'm6ac6', 'm6ac7'], ['m6ac3', 'm6ac6', 'm6ac7'], 'HH_wealth1')

# Merge wealth data from durable appliances (6B)
# Define the required columns (household identifiers + selected variables)
# m6bc3: quantity of the durable appliance, m6bc6: durable appliance value at current price
df = merge_wealth(df, 'muc6b.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm6bc3', 'm6bc6'], ['m6bc3', 'm6bc6'], 'HH_wealth2')

# Fill missing values with zero and compute total household wealth
df[['HH_wealth1', 'HH_wealth2']] = df[['HH_wealth1', 'HH_wealth2']].fillna(0)
df['HH_Wealth'] = df['HH_wealth1'] + df['HH_wealth2']

# ******************** CALCULATE TOTAL HOUSEHOLD CONSUMPTION (EXPENDITURES) *******************************************
# Function to process and merge expenditure data
def merge_expenditure(df, file_name, columns, expense_cols, new_col_name):
    data = pd.read_csv(os.path.join(data_path, file_name))[columns] # Load the expenditure datasets with selected columns
    exp = data.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])[expense_cols].sum().reset_index() # Group the columns by household ID
    exp[new_col_name] = exp[expense_cols].sum(axis=1) # Take the sum of the expenses by households
    exp = exp.drop_duplicates(subset=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], keep='first') # Remove any duplicates, only keep a unique expense values for each household ID
    return df.merge(exp, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left') # Merge the datasets to the dataframe df

In [238]:
# Merge all expenditure files

# Load household expenditure of food and drinks during holidays (5A1)
# Define the required columns (household identifiers + selected variables)
# m5a1c2b: expense bought, m5a1c3b: expense self supplied or received
df = merge_expenditure(df, 'muc5a1.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5a1c2b', 'm5a1c3b'], ['m5a1c2b', 'm5a1c3b'], 'HH_exp1')

# Load household daily expenditure on food and drinks (5A2)
# Define the required columns (household identifiers + selected variables)
# m5a2c6: expense bought, m5a2c10: expense self supplied or received
df = merge_expenditure(df, 'muc5a2.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5a2c6', 'm5a2c10'], ['m5a2c6', 'm5a2c10'], 'HH_exp2')

# Load household daily expenditure on nonfood and others
# Define the required columns (household identifiers + selected variables)
# m5b1c4: expense recieved, m5b1c5: annual expense
df = merge_expenditure(df, 'muc5b1.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5b1c4', 'm5b1c5'], ['m5b1c4', 'm5b1c5'], 'HH_exp3')

# Load household annual consumption expenditure
# Define the required columns (household identifiers + selected variables)
# m5b2c2: expense bought, m5b2c3: expense self supplied or received
df = merge_expenditure(df, 'muc5b2.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5b2c2', 'm5b2c3'], ['m5b2c2', 'm5b2c3'], 'HH_exp4')

# Load other spending that is considered as household expenditure
# Define the required columns (household identifiers + selected variables)
# m5b3c2: annual expense
df = merge_expenditure(df, 'muc5b3.csv', ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm5b3c2'], ['m5b3c2'], 'HH_exp5')

# Load household's accomodation expenditure
# Define the required columns (household identifiers + selected variables)
# m7c32: annual water expense, m7c36: annual electricity expense , m7c39: annual garbage collection expense (Expenses)
muc7 = pd.read_csv(os.path.join(data_path, 'muc7.csv'))
columns7 = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'm7c32', 'm7c36', 'm7c39']
muc7 = muc7[columns7]
muc7['HH_exp6'] = muc7[['m7c32', 'm7c36', 'm7c39']].sum(axis=1)
muc7.fillna(0, inplace=True)
df = df.merge(muc7, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso'], how='left')

In [239]:
# Aggregate total household consumption expenditure (sum of HH_exp1 to HH_exp6)
df['HH_consumption'] = (
    df['HH_exp1'] + df['HH_exp2'] + df['HH_exp3'] +
    df['HH_exp4'] + df['HH_exp5'] + df['HH_exp6']
)

# Calculate the average household consumption 
df['HH_consumption_avr'] = df['HH_consumption']/ df['hsize']

# Rename 'm1ac5' to 'age'
df.rename(columns={'m1ac5': 'age'}, inplace=True)

In [240]:
df.columns

Index(['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv', 'hsize', 'm1ac2',
       'm1ac3', 'age', 'm4ac6', 'm4ac7', 'm4ac8', 'm4ac11', 'm4ac12f',
       'm4ac16', 'm4ac17', 'm4ac18', 'm4ac21', 'm4ac22f', 'm4ac25',
       'indi_income', 'hh_income', 'total_hours_first', 'total_hours_second',
       'indi_work_hours', 'hh_work_hours', 'HH_wealth1', 'HH_wealth2',
       'HH_Wealth', 'm5a1c2b', 'm5a1c3b', 'HH_exp1', 'm5a2c6', 'm5a2c10',
       'HH_exp2', 'm5b1c4', 'm5b1c5', 'HH_exp3', 'm5b2c2', 'm5b2c3', 'HH_exp4',
       'm5b3c2', 'HH_exp5', 'm7c32', 'm7c36', 'm7c39', 'HH_exp6',
       'HH_consumption', 'HH_consumption_avr'],
      dtype='object')

In [241]:
# Compute the log of income, handle missing values
df['log_income'] = np.log(df['hh_income'].replace(0, np.nan))  # replace 0 with NaN to avoid taking log(0)

# Group by age and calculate the average log income for each age group
log_income_by_age = df.groupby('age')['log_income'].mean()

# Exponentiate the average log income to get G_t for each age
G_t = np.exp(log_income_by_age)

# Create a new DataFrame with age and associated G_t
Gt_df = pd.DataFrame({'age': log_income_by_age.index, 'G_t': G_t})
# Reset index for G_t_df to make 'age' a column instead of an index
Gt_df.reset_index(drop=True, inplace=True)

# Compute the average total working hours for each age group
#work_hours_by_age = df.groupby('age')['hh_work_hours'].mean()

# Merge the work hours data with G_t_df based on age
#Gt_df = Gt_df.merge(work_hours_by_age, on='age', how='left')
Gt_df

,age,G_t
0,25,9588.999868
1,26,9827.771315
2,27,10759.763808
3,28,9407.035797
4,29,12887.171287
...,...,...
67,92,62900.859255
68,93,NaN
69,94,NaN
70,95,NaN


In [242]:
# Create household size column
muc4a['hsize'] = muc4a.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['matv'].transform('max')

# Select the required columns
columns = ['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv', 'm4ac6', 'm4ac7', 'm4ac8', 'm4ac11', 
             'm4ac12f', 'm4ac16', 'm4ac17', 'm4ac18', 'm4ac21', 'm4ac22f', 'm4ac25']
muc4aa = muc4a[columns].copy()

# Define income-related columns and filter existing ones
income_cols = ['m4ac11', 'm4ac12f', 'm4ac21', 'm4ac22f', 'm4ac25']
income_cols = [col for col in income_cols if col in muc4aa.columns]

# Compute individual income (handle missing columns)
muc4aa['indi_income'] = muc4aa[income_cols].sum(axis=1) if income_cols else 0

# Compute total household income
hh_income = muc4aa.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'])['indi_income'].sum().reset_index()

# Merge total household income
muc4aa = muc4aa.merge(hh_income, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'], how='left', suffixes=('', '_total'))

# If we have a duplicate column, drop the old column to avoid confusion
if 'indi_income_total' in muc4aa.columns:
    muc4aa.drop(columns=['indi_income'], inplace=True)

# Rename merged income column to avoid ambiguity
muc4aa.rename(columns={'indi_income_total': 'indi_income'}, inplace=True)

# Filter out individuals with no income (indi_income <= 0)
muc4aa = muc4aa[muc4aa['indi_income'] > 0]

# Create a new column 'working_individuals' to count the number of working individuals per household
# Group by household and count the number of non-zero income individuals (working individuals)
muc4aa['working_individuals'] = muc4aa.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['matv'].transform('count')

# If you want to keep 'hsize' as the number of working individuals per household, calculate that based on 'working_individuals'
muc4aa['hsize_working'] = muc4aa.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso'])['working_individuals'].transform('max')

# Define work-related columns and filter existing ones
work_cols = ['m4ac6', 'm4ac7', 'm4ac8', 'm4ac16', 'm4ac17', 'm4ac18']
work_cols = [col for col in work_cols if col in muc4aa.columns]

# Fill NaN values with 0 to avoid multiplication errors
muc4aa[work_cols] = muc4aa[work_cols].fillna(0)

# Compute total working hours for first and secondary jobs
muc4aa['total_hours_first'] = muc4aa['m4ac6'] * muc4aa['m4ac7'] * muc4aa['m4ac8']
muc4aa['total_hours_second'] = muc4aa['m4ac16'] * muc4aa['m4ac17'] * muc4aa['m4ac18']

# Compute total individual working hours
muc4aa['indi_work_hours'] = muc4aa['total_hours_first'] + muc4aa['total_hours_second']

# Filter out individuals with no working hour (indi_work_hours <= 0)
muc4aa = muc4aa[muc4aa['indi_work_hours'] > 0]

# Compute total household working hours
hh_work_hours = muc4aa.groupby(['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'])['indi_work_hours'].sum().reset_index()
hh_work_hours.rename(columns={'indi_work_hours': 'hh_work_hours'}, inplace=True)

# Merge total household working hours
muc4aa = muc4aa.merge(hh_work_hours, on=['tinh', 'huyen', 'xa', 'diaban', 'hoso', 'matv'], how='left')

# Fill any remaining NaN values
muc4aa.fillna(0, inplace=True)
muc4aa

,tinh,huyen,xa,diaban,hoso,matv,m4ac6,m4ac7,m4ac8,m4ac11,...,m4ac21,m4ac22f,m4ac25,indi_income,working_individuals,hsize_working,total_hours_first,total_hours_second,indi_work_hours,hh_work_hours
0,101,1,3,1,13,2,12.0,26.0,8.0,17632.0,...,0.0,0,0.0,18550.0,1,1,2496.0,0.0,2496.0,2496.0
1,101,1,3,1,14,1,12.0,26.0,8.0,39528.0,...,0.0,0,0.0,42351.0,2,2,2496.0,0.0,2496.0,2496.0
2,101,1,3,1,14,2,6.0,28.0,8.0,14117.0,...,0.0,0,0.0,14117.0,2,2,1344.0,480.0,1824.0,1824.0
3,101,1,3,1,15,2,12.0,26.0,8.0,39528.0,...,0.0,0,0.0,42351.0,1,1,2496.0,0.0,2496.0,2496.0
4,101,1,9,19,19,3,12.0,23.0,8.0,22040.0,...,0.0,0,0.0,22040.0,1,1,2208.0,0.0,2208.0,2208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,823,13,11,4,13,2,9.0,22.0,6.0,28527.0,...,0.0,0,0.0,29867.0,2,2,1188.0,180.0,1368.0,1368.0
9354,823,13,12,25,15,3,10.0,25.0,8.0,19912.0,...,0.0,0,0.0,19978.0,3,3,2000.0,0.0,2000.0,2000.0
9355,823,13,12,25,15,4,11.0,20.0,8.0,5310.0,...,0.0,0,0.0,5310.0,3,3,1760.0,0.0,1760.0,1760.0
9356,823,13,12,25,15,5,2.0,25.0,6.0,2389.0,...,0.0,0,0.0,2389.0,3,3,300.0,0.0,300.0,300.0


In [245]:
muc4aa['hsize_working']

0       1
1       2
2       2
3       1
4       1
       ..
9353    2
9354    3
9355    3
9356    3
9357    1
Name: hsize_working, Length: 9358, dtype: int64

In [244]:
'''
# Scatter plot with trend line
plt.figure(figsize=(10, 6))
sns.regplot(x=Gt_df['G_t'], y=Gt_df['hh_work_hours'], 
            scatter_kws={'s': 10}, line_kws={'color': 'red'})

plt.title('Scatter plot of Total Household Working Hours vs. Total Household Income with Trend Line')
plt.xlabel('G_t (Exponentiated Income)')
plt.ylabel('Total Household Working Hours')
plt.show()
'''

"\n# Scatter plot with trend line\nplt.figure(figsize=(10, 6))\nsns.regplot(x=Gt_df['G_t'], y=Gt_df['hh_work_hours'], \n            scatter_kws={'s': 10}, line_kws={'color': 'red'})\n\nplt.title('Scatter plot of Total Household Working Hours vs. Total Household Income with Trend Line')\nplt.xlabel('G_t (Exponentiated Income)')\nplt.ylabel('Total Household Working Hours')\nplt.show()\n"